In [1]:
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from pylab import rcParams
import statsmodels.api as sm

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# function to split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
df_sales = pd.read_csv('../data/sales_train_v2.csv')
# correct the date
df_sales['date'] = pd.to_datetime(df_sales['date'], dayfirst=True)
# to handle months with no data, first create an empty dataframe with all the dateblocks
df_sales_month_empty = pd.DataFrame({'date_block_num':np.arange(34)})


In [4]:
def LSTM_Predict(df_sales, shop_id):
    # get the monthly item count
    df_sales_month = df_sales.loc[(df_sales['shop_id']==shop_id)].groupby(['date_block_num'])['item_cnt_day'].sum().reset_index(name='item_tot_month')
    # merge into the empty dataframe to have 0's for months with no sales
    df_sales_month = pd.merge(df_sales_month_empty,df_sales_month, how='left', on='date_block_num').fillna(0)
    # define the raw input sequence
    raw_seq = df_sales_month['item_tot_month'].to_list()
    # choose a number of time steps
    n_steps = 5
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    n_train = len(raw_seq) - n_steps

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    # define model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=200, verbose=0)
    # predict using the last n_steps inputs
    x_input = array(raw_seq[n_train:])
    x_input = x_input.reshape((1, n_steps, n_features))
    # print the prediction for November 2015
    yhat = model.predict(x_input, verbose=0)
    return yhat

In [5]:
LSTM_Predict(df_sales, 59)

array([[726.3396]], dtype=float32)

In [6]:
df_sales_shop_total = df_sales.groupby(['shop_id'])['item_cnt_day'].sum().reset_index(name='item_cnt_shop_id')
df_sales_shop_total = df_sales_shop_total.sort_values(by='item_cnt_shop_id',ascending=False)
df_sales_shop_total.head(10)

,shop_id,item_cnt_shop_id
31,31,310777.0
25,25,241920.0
54,54,185790.0
28,28,184557.0
42,42,144934.0
57,57,141107.0
27,27,136657.0
6,6,100489.0
58,58,81734.0
46,46,78990.0


In [7]:
for index, row in df_sales_shop_total.head(10).iterrows():
     
    yhat = LSTM_Predict(df_sales, row['shop_id'])
    print ('Prediction for shop ' + str(row['shop_id']) + ' is ' + str(yhat[0]))

Prediction for shop 31.0 is [7159.106]
Prediction for shop 25.0 is [7143.699]
Prediction for shop 54.0 is [0.01587937]
Prediction for shop 28.0 is [3226.337]
Prediction for shop 42.0 is [4149.655]
Prediction for shop 57.0 is [2556.5422]
Prediction for shop 27.0 is [1037.9011]
Prediction for shop 6.0 is [477.75122]
Prediction for shop 58.0 is [1296.3966]
Prediction for shop 46.0 is [1567.5078]
